## PER + PBR 합성 전략
- PBR < 1 : 자산 대비 저평가
- PER < 10 : 수익 대비 저평가

### 맞는 데이터 가져오고 merge 하기

In [34]:
from pykrx import stock
import pandas as pd

In [35]:
year = '2023'
first_df = stock.get_market_cap_by_ticker(f'{year}0102')
first_df = first_df[['종가','시가총액']].rename(columns={'종가':'시가'})
first_df.head()

,시가,시가총액
티커,,
005930,55500,331322931525000
373220,446000,104364000000000
207940,827000,58860898000000
000660,75700,55109779030500
051910,604000,42637775172000


In [36]:
first_df.describe()

,시가,시가총액
count,2690.000000,2.690000e+03
mean,19262.999257,7.713959e+11
std,51201.159850,7.256943e+12
min,49.000000,1.178126e+09
25%,2636.250000,5.396002e+10
50%,6235.000000,1.092750e+11
75%,14925.000000,2.592859e+11
max,827000.000000,3.313229e+14


In [37]:
first_df = first_df.sort_values('시가총액')
first_df.head()

,시가,시가총액
티커,,
322190,132,1178126004
267060,181,1654666524
267810,1010,1804870000
288490,49,1869203000
215050,912,1934959392


In [38]:
bin = 3
labels = ['소형주', '중형주', '대형주']
first_df['symbol_group'] = pd.cut(first_df.reset_index().index, bins=bin, labels=labels)
first_df.head() # symbol_group 

,시가,시가총액,symbol_group
티커,,,
322190,132,1178126004,소형주
267060,181,1654666524,소형주
267810,1010,1804870000,소형주
288490,49,1869203000,소형주
215050,912,1934959392,소형주


### read PER + PBR from pykrx

In [39]:
year = '2023'
second_df = stock.get_market_fundamental_by_ticker(f'{year}0102')
second_df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
095570,8075,3.35,0.71,1707,4.72,270
006840,45961,0.00,0.35,0,1.23,200
027410,16393,6.02,0.25,684,2.67,110
282330,46849,23.63,4.31,8547,1.49,3000
138930,28745,2.69,0.22,2341,8.89,560


In [43]:
second_df = second_df[['PER', 'PBR']]
second_df.head()

,PER,PBR
티커,,
095570,3.35,0.71
006840,0.00,0.35
027410,6.02,0.25
282330,23.63,4.31
138930,2.69,0.22


### 해당년도 마지막 거래 종가

In [42]:
third_df = stock.get_market_ohlcv_by_ticker(f'{year}1228')
third_df.head()

,시가,고가,저가,종가,거래량,거래대금,등락률
티커,,,,,,,
095570,4860,5020,4860,4980,144515,719851745,1.53
006840,17170,17170,16910,17100,3459,58863090,0.59
027410,3785,3835,3745,3810,85988,326300075,0.40
282330,129300,131400,128500,131300,72995,9499511500,2.02
138930,7080,7160,7020,7140,1267670,8991939180,0.85


In [44]:
third_df = third_df[['종가']]
third_df.head()

,종가
티커,
095570,4980
006840,17100
027410,3810
282330,131300
138930,7140


#### merge

In [46]:
concat_df = pd.concat([first_df,second_df,third_df],axis=1)
concat_df
concat_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2708 entries, 322190 to 453340
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   시가            2690 non-null   float64 
 1   시가총액          2690 non-null   float64 
 2   symbol_group  2690 non-null   category
 3   PER           914 non-null    float64 
 4   PBR           914 non-null    float64 
 5   종가            953 non-null    float64 
dtypes: category(1), float64(5)
memory usage: 129.7+ KB


In [47]:
concat_dropna_df = concat_df.dropna()
concat_dropna_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 002787 to 005930
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   시가            905 non-null    float64 
 1   시가총액          905 non-null    float64 
 2   symbol_group  905 non-null    category
 3   PER           905 non-null    float64 
 4   PBR           905 non-null    float64 
 5   종가            905 non-null    float64 
dtypes: category(1), float64(5)
memory usage: 43.4+ KB


In [48]:
concat_dropna_df['수익률'] = concat_dropna_df['종가'] / concat_dropna_df['시가']
concat_dropna_df.describe()

/tmp/ipykernel_35293/3844793185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_dropna_df['수익률'] = concat_dropna_df['종가'] / concat_dropna_df['시가']


,시가,시가총액,PER,PBR,종가,수익률
count,905.000000,9.050000e+02,905.000000,905.000000,905.000000,905.000000
mean,38305.522652,1.918690e+12,19.922796,0.991735,38432.748066,1.128948
std,81887.812583,1.241832e+13,222.605209,1.420876,77110.309587,0.729142
min,199.000000,3.773542e+09,0.000000,0.000000,165.000000,0.065122
25%,3745.000000,8.191163e+10,0.000000,0.330000,3910.000000,0.864247
50%,11000.000000,2.174614e+11,4.940000,0.580000,11550.000000,1.000000
75%,34550.000000,7.277092e+11,12.070000,1.160000,37000.000000,1.192810
max,827000.000000,3.313229e+14,6650.000000,16.710000,760000.000000,11.974359


In [53]:
conditions = (concat_dropna_df['PER'] >= 2.5) & (concat_dropna_df['PER'] <= 10)
concat_dropna_df[conditions].sort_values('PBR').head(5)


,시가,시가총액,symbol_group,PER,PBR,종가,수익률
티커,,,,,,,
012630,5230.0,3.124492e+11,대형주,2.53,0.12,6890.0,1.317400
003830,117000.0,1.553760e+11,중형주,5.39,0.18,105700.0,0.903419
085620,2915.0,5.160022e+11,대형주,4.83,0.20,4765.0,1.634648
005720,4080.0,2.184594e+11,대형주,7.83,0.21,4165.0,1.020833
058650,104000.0,4.160000e+11,대형주,8.33,0.21,108000.0,1.038462
